In [48]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
import time
import glob

import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Globally allows loading truncated images

from tqdm import tqdm_notebook

In [49]:
label_index = pd.read_csv("chinese_herbal_medicine.csv")
label_index = {k:v for v,k in label_index["category"].to_dict().items()}

In [50]:
train_path = glob.glob('./train_set/*/*.jpg')
np.random.shuffle(train_path)

train_label = [label_index[x.split('/')[-2]] for x in train_path]

test_path = glob.glob('./test_set/*.jpg')
test_path.sort()

In [51]:
len(train_path)

8525

In [66]:
for path in train_path:
    try:
        img = Image.open(path).convert('RGB')
        img.size
    except:
        train_path.remove(path)

In [67]:
len(train_path)

8122

In [68]:
train_label = [label_index[x.split('/')[-2]] for x in train_path]

In [69]:
test_path[:10]

['./test_set/006f490b-c352-414c-be06-0e14a39eb3ee.jpg',
 './test_set/007d7487-413d-4d11-9df1-1c40651f6428.jpg',
 './test_set/00b15eef-38c6-42e0-b1ed-5bbd34d9f20f.jpg',
 './test_set/00d3d2ea-2f66-40b7-b402-b2bcd36cf776.jpg',
 './test_set/01207b72-00c6-43c4-aa14-85e76c138022.jpg',
 './test_set/01385aa6-d1ca-4aa9-831f-a63ab7a02d66.jpg',
 './test_set/01631a84-1657-438e-94bc-3e5de97a7907.jpg',
 './test_set/0172672f-e02d-4809-a40c-b1ec16b25281.jpg',
 './test_set/0185e720-8592-44cb-8332-9e39c9e490ba.jpg',
 './test_set/018b4cd5-77ef-495d-a641-1df43725aa3b.jpg']

In [70]:
len(label_index)

54

In [71]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""


    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1)

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
            losses.update(loss.item(), input.size(0))
            top1.update(acc, input.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))
        return top1

def predict(test_loader, model, tta=10):
    # switch to evaluate mode
    model.eval()
    
    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):
                input = input.cuda()
                target = target.cuda()

                # compute output
                output = model(input)
                output = F.softmax(output, dim=1)
                output = output.data.cpu().numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)
    
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses, top1)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

        acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
        top1.update(acc, input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            progress.pr2int(i)

In [86]:
class XFDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        try:
            img = Image.open(self.img_path[index]).convert('RGB')
        except:
            img = Image.new('RGB', (224, 224), color='white')

        label = torch.from_numpy(np.array(self.img_label[index]))
        if self.transform is not None:
            img = self.transform(img)
        
        return img, label
    
    def __len__(self):
        return len(self.img_path)

In [82]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import timm
model = timm.create_model('resnet18', pretrained=True, num_classes=len(label_index))
model = model.cuda()

In [83]:
train_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[:-500], train_label[:-500], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=10, shuffle=True, num_workers=4, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[-500:], train_label[-500:], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=10, shuffle=False, num_workers=4, pin_memory=True
)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
best_acc = 0.0
for epoch in range(20):
    scheduler.step()
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch)
    val_acc = validate(val_loader, model, criterion)
    
    if val_acc.avg.item() > best_acc:
        best_acc = round(val_acc.avg.item(), 2)

Epoch:  0
[  0/763]	Time  0.280 ( 0.280)	Loss 4.0691e+00 (4.0691e+00)	Acc@1   0.00 (  0.00)
[100/763]	Time  0.028 ( 0.029)	Loss 3.3308e+00 (3.7872e+00)	Acc@1  10.00 (  7.43)
[200/763]	Time  0.120 ( 0.029)	Loss 2.8703e+00 (3.4719e+00)	Acc@1  30.00 ( 14.08)
[300/763]	Time  0.171 ( 0.028)	Loss 3.0330e+00 (3.2177e+00)	Acc@1  30.00 ( 20.47)
[400/763]	Time  0.025 ( 0.028)	Loss 1.8518e+00 (3.0140e+00)	Acc@1  50.00 ( 24.14)
[500/763]	Time  0.025 ( 0.028)	Loss 1.8098e+00 (2.8778e+00)	Acc@1  70.00 ( 27.25)
[600/763]	Time  0.028 ( 0.027)	Loss 2.1014e+00 (2.7492e+00)	Acc@1  60.00 ( 30.25)
[700/763]	Time  0.026 ( 0.027)	Loss 2.4719e+00 (2.6548e+00)	Acc@1  30.00 ( 32.52)


/tmp/ipykernel_3559365/1274669909.py:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):


  0%|          | 0/50 [00:00<?, ?it/s]

 * Acc@1 51.200
Epoch:  1
[  0/763]	Time  0.268 ( 0.268)	Loss 1.3543e+00 (1.3543e+00)	Acc@1  60.00 ( 60.00)
[100/763]	Time  0.022 ( 0.032)	Loss 1.6205e+00 (1.8530e+00)	Acc@1  50.00 ( 49.50)
[200/763]	Time  0.028 ( 0.030)	Loss 1.0775e+00 (1.8112e+00)	Acc@1  70.00 ( 51.04)
[300/763]	Time  0.024 ( 0.029)	Loss 1.0627e+00 (1.7675e+00)	Acc@1  90.00 ( 52.19)
[400/763]	Time  0.024 ( 0.029)	Loss 1.8049e+00 (1.7593e+00)	Acc@1  40.00 ( 52.69)
[500/763]	Time  0.027 ( 0.029)	Loss 1.3464e+00 (1.7254e+00)	Acc@1  40.00 ( 53.43)
[600/763]	Time  0.028 ( 0.028)	Loss 2.3905e+00 (1.7023e+00)	Acc@1  40.00 ( 53.76)
[700/763]	Time  0.024 ( 0.028)	Loss 1.9786e+00 (1.6916e+00)	Acc@1  60.00 ( 54.14)


  0%|          | 0/50 [00:00<?, ?it/s]

...

 * Acc@1 77.200
Epoch:  18
[  0/763]	Time  0.291 ( 0.291)	Loss 1.8693e-01 (1.8693e-01)	Acc@1  90.00 ( 90.00)
[100/763]	Time  0.025 ( 0.031)	Loss 1.7509e-01 (1.9842e-01)	Acc@1  90.00 ( 93.96)
[200/763]	Time  0.025 ( 0.030)	Loss 3.7665e-01 (1.6763e-01)	Acc@1  80.00 ( 94.88)
[300/763]	Time  0.026 ( 0.029)	Loss 2.0604e-02 (1.6482e-01)	Acc@1 100.00 ( 95.12)
[400/763]	Time  0.025 ( 0.028)	Loss 2.3494e-02 (1.6603e-01)	Acc@1 100.00 ( 95.06)
[500/763]	Time  0.025 ( 0.028)	Loss 4.0113e-02 (1.5512e-01)	Acc@1 100.00 ( 95.49)
[600/763]	Time  0.020 ( 0.028)	Loss 1.6110e-01 (1.5024e-01)	Acc@1 100.00 ( 95.76)
[700/763]	Time  0.027 ( 0.028)	Loss 7.6947e-02 (1.5441e-01)	Acc@1 100.00 ( 95.61)


  0%|          | 0/50 [00:00<?, ?it/s]

 * Acc@1 77.600
Epoch:  19
[  0/763]	Time  0.311 ( 0.311)	Loss 2.1786e-01 (2.1786e-01)	Acc@1  90.00 ( 90.00)
[100/763]	Time  0.029 ( 0.030)	Loss 7.6430e-02 (1.2448e-01)	Acc@1 100.00 ( 96.24)
[200/763]	Time  0.031 ( 0.028)	Loss 5.2715e-03 (1.0296e-01)	Acc@1 100.00 ( 97.26)
[300/763]	Time  0.023 ( 0.028)	Loss 6.3700e-02 (1.0464e-01)	Acc@1 100.00 ( 97.21)
[400/763]	Time  0.025 ( 0.028)	Loss 2.3657e-03 (1.0401e-01)	Acc@1 100.00 ( 97.21)
[500/763]	Time  0.025 ( 0.028)	Loss 4.1642e-02 (1.0472e-01)	Acc@1 100.00 ( 97.11)
[600/763]	Time  0.025 ( 0.028)	Loss 2.3447e-02 (1.0926e-01)	Acc@1 100.00 ( 96.94)
[700/763]	Time  0.029 ( 0.028)	Loss 1.1835e-01 (1.1077e-01)	Acc@1  90.00 ( 96.78)


  0%|          | 0/50 [00:00<?, ?it/s]

 * Acc@1 78.200


In [84]:
len(train_path)

8122

In [89]:
test_path = list("./test_set/" + pd.read_csv("example_A.csv")["ImageID"])

test_loader = torch.utils.data.DataLoader(
    XFDataset(test_path, [0] * len(test_path), 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=40, shuffle=False, num_workers=4, pin_memory=True
)

val_label = pd.DataFrame()
val_label['ImageID'] = [x.split('/')[-1] for x in test_path]
val_label['label'] = predict(test_loader, model, 1).argmax(1)
val_label.to_csv('submit.csv', index=None)

/tmp/ipykernel_3559365/1274669909.py:82: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):


  0%|          | 0/46 [00:00<?, ?it/s]

In [90]:
val_label

,ImageID,label
0,006f490b-c352-414c-be06-0e14a39eb3ee.jpg,5
1,007d7487-413d-4d11-9df1-1c40651f6428.jpg,15
2,00b15eef-38c6-42e0-b1ed-5bbd34d9f20f.jpg,18
3,00d3d2ea-2f66-40b7-b402-b2bcd36cf776.jpg,1
4,01207b72-00c6-43c4-aa14-85e76c138022.jpg,53
...,...,...
1820,ff3d0831-0110-4106-b76a-c9f5c8291a18.jpg,18
1821,ff5c00d4-91eb-4347-84c2-2824654eb625.jpg,25
1822,ff74d365-3be6-49e9-878f-806e8c8b83de.jpg,34
1823,ff864ef5-5cb2-4d63-a6d5-d6f8980f6f23.jpg,37
